In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers import MaxPooling2D, concatenate
from keras.layers import Lambda
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential
from keras.models import Model
from keras.optimizers import Adam
import keras.models
from keras import losses
from keras.utils import to_categorical
import keras.backend as K

import matplotlib.pyplot as plt

import numpy as np

class AdversarialAutoencoder():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 10

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the encoder / decoder
        self.encoder = self.build_encoder()
        self.decoder = self.build_decoder()

        img = Input(shape=self.img_shape)
        # The generator takes the image, encodes it and reconstructs it
        # from the encoding
        encoded_repr = self.encoder(img)
        reconstructed_img = self.decoder(encoded_repr)

        # For the adversarial_autoencoder model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator determines validity of the encoding
        validity = self.discriminator(encoded_repr)

        # The adversarial_autoencoder model  (stacked generator and discriminator)
        self.adversarial_autoencoder = Model(img, [reconstructed_img, validity])
        self.adversarial_autoencoder.compile(loss=['mse', 'binary_crossentropy'],
            loss_weights=[0.999, 0.001],
            optimizer=optimizer)


    def build_encoder(self):
        # Encoder

        img = Input(shape=self.img_shape)

        h = Flatten()(img)
        h = Dense(512)(h)
        h = LeakyReLU(alpha=0.2)(h)
        h = Dense(512)(h)
        h = LeakyReLU(alpha=0.2)(h)
        mu = Dense(self.latent_dim)(h)
        log_var = Dense(self.latent_dim)(h)
        
        merge_layer = Lambda(lambda p: p[0] + K.random_normal(K.shape(p[0])) * K.exp(p[1] / 2),
                             output_shape=lambda p: p[0])
        
#         latent_repr = Merge([mu, log_var],
#                 mode=lambda p: p[0] + K.random_normal(K.shape(p[0])) * K.exp(p[1] / 2),
#                 output_shape=lambda p: p[0])

        latent_repr = merge_layer([mu, log_var])
        
        return Model(inputs=img, outputs=latent_repr)

    def build_decoder(self):

        model = Sequential()

        model.add(Dense(512, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        z = Input(shape=(self.latent_dim,))
        img = model(z)

        return Model(z, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Dense(512, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation="sigmoid"))
        model.summary()

        encoded_repr = Input(shape=(self.latent_dim, ))
        validity = model(encoded_repr)

        return Model(encoded_repr, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            latent_fake = self.encoder.predict(imgs)
            latent_real = np.random.normal(size=(batch_size, self.latent_dim))

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(latent_real, valid)
            d_loss_fake = self.discriminator.train_on_batch(latent_fake, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator
            g_loss = self.adversarial_autoencoder.train_on_batch(imgs, [imgs, valid])

            # Plot the progress
            print ("%d [D loss: %f, acc: %.2f%%] [G loss: %f, mse: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[0], g_loss[1]))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5

        z = np.random.normal(size=(r*c, self.latent_dim))
        gen_imgs = self.decoder.predict(z)

        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/mnist_%d.png" % epoch)
        plt.close()

    def save_model(self):

        def save(model, model_name):
            model_path = "saved_model/%s.json" % model_name
            weights_path = "saved_model/%s_weights.hdf5" % model_name
            options = {"file_arch": model_path,
                        "file_weight": weights_path}
            json_string = model.to_json()
            open(options['file_arch'], 'w').write(json_string)
            model.save_weights(options['file_weight'])

        save(self.generator, "aae_generator")
        save(self.discriminator, "aae_discriminator")


if __name__ == '__main__':
    aae = AdversarialAutoencoder()
aae.train(epochs=20000, batch_size=32, sample_interval=200)

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               5632      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 137,217
Trainable params: 137,217
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dens

/home/rytse/anaconda3/envs/b3s/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.709589, acc: 42.19%] [G loss: 0.940871, mse: 0.941075]
1 [D loss: 0.692729, acc: 50.00%] [G loss: 0.925801, mse: 0.925996]
2 [D loss: 0.667669, acc: 67.19%] [G loss: 0.898998, mse: 0.899074]
3 [D loss: 0.569789, acc: 78.12%] [G loss: 0.864179, mse: 0.863917]
4 [D loss: 0.417968, acc: 78.12%] [G loss: 0.774473, mse: 0.773091]
5 [D loss: 0.349137, acc: 82.81%] [G loss: 0.670047, mse: 0.667191]
6 [D loss: 0.310984, acc: 84.38%] [G loss: 0.548849, mse: 0.544495]
7 [D loss: 0.307185, acc: 87.50%] [G loss: 0.449757, mse: 0.444230]
8 [D loss: 0.270267, acc: 90.62%] [G loss: 0.365112, mse: 0.358550]
9 [D loss: 0.266469, acc: 90.62%] [G loss: 0.336189, mse: 0.329653]
10 [D loss: 0.246236, acc: 92.19%] [G loss: 0.295980, mse: 0.289375]
11 [D loss: 0.229609, acc: 96.88%] [G loss: 0.271594, mse: 0.265153]
12 [D loss: 0.211247, acc: 96.88%] [G loss: 0.299915, mse: 0.294235]
13 [D loss: 0.212009, acc: 98.44%] [G loss: 0.283543, mse: 0.278136]
14 [D loss: 0.181557, acc: 100.00%] [G loss:

124 [D loss: 0.235703, acc: 87.50%] [G loss: 0.155312, mse: 0.147980]
125 [D loss: 0.416617, acc: 78.12%] [G loss: 0.144555, mse: 0.139715]
126 [D loss: 0.247270, acc: 87.50%] [G loss: 0.158875, mse: 0.151363]
127 [D loss: 0.326883, acc: 84.38%] [G loss: 0.173222, mse: 0.168130]
128 [D loss: 0.286458, acc: 85.94%] [G loss: 0.146597, mse: 0.140409]
129 [D loss: 0.361053, acc: 81.25%] [G loss: 0.155632, mse: 0.149388]
130 [D loss: 0.302384, acc: 84.38%] [G loss: 0.153771, mse: 0.145935]
131 [D loss: 0.245911, acc: 89.06%] [G loss: 0.133143, mse: 0.127602]
132 [D loss: 0.276885, acc: 87.50%] [G loss: 0.167766, mse: 0.160148]
133 [D loss: 0.250949, acc: 89.06%] [G loss: 0.139783, mse: 0.132066]
134 [D loss: 0.258430, acc: 90.62%] [G loss: 0.148051, mse: 0.142713]
135 [D loss: 0.383372, acc: 87.50%] [G loss: 0.158680, mse: 0.152013]
136 [D loss: 0.264920, acc: 89.06%] [G loss: 0.138193, mse: 0.130897]
137 [D loss: 0.210115, acc: 93.75%] [G loss: 0.148337, mse: 0.142183]
138 [D loss: 0.22469

...

19434 [D loss: 0.311558, acc: 82.81%] [G loss: 0.054365, mse: 0.052374]
19435 [D loss: 0.359490, acc: 84.38%] [G loss: 0.045358, mse: 0.043488]
19436 [D loss: 0.407549, acc: 84.38%] [G loss: 0.063722, mse: 0.061890]
19437 [D loss: 0.327935, acc: 87.50%] [G loss: 0.054531, mse: 0.052277]
19438 [D loss: 0.540248, acc: 78.12%] [G loss: 0.055858, mse: 0.054040]
19439 [D loss: 0.266528, acc: 90.62%] [G loss: 0.052668, mse: 0.050741]
19440 [D loss: 0.307167, acc: 90.62%] [G loss: 0.055982, mse: 0.053930]
19441 [D loss: 0.463944, acc: 76.56%] [G loss: 0.054475, mse: 0.052644]
19442 [D loss: 0.356414, acc: 81.25%] [G loss: 0.046218, mse: 0.043685]
19443 [D loss: 0.355923, acc: 87.50%] [G loss: 0.057802, mse: 0.055768]
19444 [D loss: 0.343679, acc: 85.94%] [G loss: 0.045739, mse: 0.043730]
19445 [D loss: 0.411453, acc: 79.69%] [G loss: 0.050861, mse: 0.049072]
19446 [D loss: 0.416024, acc: 85.94%] [G loss: 0.049255, mse: 0.047311]
19447 [D loss: 0.426390, acc: 82.81%] [G loss: 0.055952, mse: 0.

19550 [D loss: 0.343685, acc: 82.81%] [G loss: 0.071073, mse: 0.069314]
19551 [D loss: 0.364104, acc: 82.81%] [G loss: 0.051288, mse: 0.049140]
19552 [D loss: 0.280924, acc: 93.75%] [G loss: 0.053907, mse: 0.052147]
19553 [D loss: 0.448640, acc: 81.25%] [G loss: 0.062277, mse: 0.060193]
19554 [D loss: 0.325879, acc: 87.50%] [G loss: 0.060724, mse: 0.058296]
19555 [D loss: 0.469292, acc: 78.12%] [G loss: 0.059364, mse: 0.057291]
19556 [D loss: 0.309234, acc: 89.06%] [G loss: 0.054707, mse: 0.052773]
19557 [D loss: 0.370646, acc: 84.38%] [G loss: 0.049858, mse: 0.047858]
19558 [D loss: 0.380967, acc: 85.94%] [G loss: 0.067942, mse: 0.066218]
19559 [D loss: 0.406253, acc: 85.94%] [G loss: 0.055884, mse: 0.054069]
19560 [D loss: 0.517699, acc: 78.12%] [G loss: 0.064892, mse: 0.063339]
19561 [D loss: 0.500600, acc: 76.56%] [G loss: 0.053905, mse: 0.052205]
19562 [D loss: 0.304803, acc: 92.19%] [G loss: 0.051039, mse: 0.049135]
19563 [D loss: 0.411899, acc: 85.94%] [G loss: 0.045610, mse: 0.

19674 [D loss: 0.431276, acc: 76.56%] [G loss: 0.063317, mse: 0.061325]
19675 [D loss: 0.321649, acc: 89.06%] [G loss: 0.050075, mse: 0.048031]
19676 [D loss: 0.411542, acc: 73.44%] [G loss: 0.056430, mse: 0.054747]
19677 [D loss: 0.441006, acc: 81.25%] [G loss: 0.064792, mse: 0.062591]
19678 [D loss: 0.362778, acc: 78.12%] [G loss: 0.064544, mse: 0.062687]
19679 [D loss: 0.413746, acc: 79.69%] [G loss: 0.057562, mse: 0.055629]
19680 [D loss: 0.506833, acc: 71.88%] [G loss: 0.063453, mse: 0.061221]
19681 [D loss: 0.450988, acc: 82.81%] [G loss: 0.057136, mse: 0.055099]
19682 [D loss: 0.465129, acc: 78.12%] [G loss: 0.058496, mse: 0.056646]
19683 [D loss: 0.501699, acc: 78.12%] [G loss: 0.064455, mse: 0.062782]
19684 [D loss: 0.446437, acc: 78.12%] [G loss: 0.069527, mse: 0.067810]
19685 [D loss: 0.422297, acc: 81.25%] [G loss: 0.050096, mse: 0.048281]
19686 [D loss: 0.456452, acc: 82.81%] [G loss: 0.060853, mse: 0.058928]
19687 [D loss: 0.347179, acc: 84.38%] [G loss: 0.058403, mse: 0.

19796 [D loss: 0.351005, acc: 81.25%] [G loss: 0.049840, mse: 0.047959]
19797 [D loss: 0.348752, acc: 90.62%] [G loss: 0.058841, mse: 0.057247]
19798 [D loss: 0.423230, acc: 78.12%] [G loss: 0.054786, mse: 0.053115]
19799 [D loss: 0.361334, acc: 87.50%] [G loss: 0.055751, mse: 0.053836]
19800 [D loss: 0.399424, acc: 79.69%] [G loss: 0.059202, mse: 0.057386]
19801 [D loss: 0.465139, acc: 79.69%] [G loss: 0.057115, mse: 0.055111]
19802 [D loss: 0.374800, acc: 82.81%] [G loss: 0.065431, mse: 0.063528]
19803 [D loss: 0.439388, acc: 82.81%] [G loss: 0.058440, mse: 0.056091]
19804 [D loss: 0.451460, acc: 76.56%] [G loss: 0.057936, mse: 0.056330]
19805 [D loss: 0.407419, acc: 85.94%] [G loss: 0.056668, mse: 0.054404]
19806 [D loss: 0.498899, acc: 75.00%] [G loss: 0.058431, mse: 0.056537]
19807 [D loss: 0.437903, acc: 81.25%] [G loss: 0.050579, mse: 0.048719]
19808 [D loss: 0.412424, acc: 79.69%] [G loss: 0.051795, mse: 0.050179]
19809 [D loss: 0.438443, acc: 81.25%] [G loss: 0.060938, mse: 0.

19913 [D loss: 0.392394, acc: 84.38%] [G loss: 0.060519, mse: 0.058763]
19914 [D loss: 0.524539, acc: 79.69%] [G loss: 0.048083, mse: 0.046165]
19915 [D loss: 0.397563, acc: 84.38%] [G loss: 0.057914, mse: 0.055804]
19916 [D loss: 0.492480, acc: 76.56%] [G loss: 0.063691, mse: 0.062093]
19917 [D loss: 0.286926, acc: 90.62%] [G loss: 0.053882, mse: 0.051555]
19918 [D loss: 0.471846, acc: 79.69%] [G loss: 0.057197, mse: 0.055201]
19919 [D loss: 0.455052, acc: 78.12%] [G loss: 0.046348, mse: 0.044169]
19920 [D loss: 0.441931, acc: 79.69%] [G loss: 0.055003, mse: 0.053172]
19921 [D loss: 0.468119, acc: 84.38%] [G loss: 0.051240, mse: 0.049689]
19922 [D loss: 0.353939, acc: 82.81%] [G loss: 0.052710, mse: 0.050834]
19923 [D loss: 0.461919, acc: 78.12%] [G loss: 0.054279, mse: 0.052611]
19924 [D loss: 0.394767, acc: 82.81%] [G loss: 0.041216, mse: 0.039253]
19925 [D loss: 0.285835, acc: 89.06%] [G loss: 0.059938, mse: 0.057600]
19926 [D loss: 0.366219, acc: 84.38%] [G loss: 0.050077, mse: 0.

In [2]:
aae.save_model()

AttributeError: 'AdversarialAutoencoder' object has no attribute 'generator'